In [42]:
import numpy as np
import pandas as pd

import os

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [38]:
df = pd.read_csv(r'D:\iNeuron\Complete Project\Fraud Detection\Credit-Card_Fraud_Detection\artifact\data_ingestion\2023-05-02_16-13-40\train\transactions')

In [33]:
df.head()

,Unnamed: 0,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO
0,758660,758660,2018-06-19 04:56:56,1410,5529,101.87,6843416,79,0,0
1,1000785,1000785,2018-07-14 09:54:39,3964,4019,95.11,9021279,104,0,0
2,328476,328476,2018-05-05 07:48:59,611,5056,43.32,2965739,34,0,0
3,1528648,1528648,2018-09-07 12:08:45,3416,763,137.22,13781325,159,0,0
4,97625,97625,2018-04-11 07:12:45,4384,1777,119.19,889965,10,0,0


In [52]:
def extract_necessary_and_drop_unncessary_columns(data):
    try:
        data['TX_DATETIME']=pd.to_datetime(data['TX_DATETIME'])
        data['day'] = data['TX_DATETIME'].dt.day
        data['month'] = data['TX_DATETIME'].dt.month

        drop_columns = ['Unnamed: 0', 'TX_FRAUD_SCENARIO','TX_DATETIME']
        data.drop(drop_columns, axis = 1, inplace = True)

        return data
    except Exception as e:
        raise e

In [39]:
new_df, columns = extract_necessary_and_drop_unncessary_columns(df)

In [40]:
new_df.head()

,TRANSACTION_ID,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,day,month
0,758660,1410,5529,101.87,6843416,79,0,19,6
1,1000785,3964,4019,95.11,9021279,104,0,14,7
2,328476,611,5056,43.32,2965739,34,0,5,5
3,1528648,3416,763,137.22,13781325,159,0,7,9
4,97625,4384,1777,119.19,889965,10,0,11,4


In [41]:
columns

Index(['TRANSACTION_ID', 'CUSTOMER_ID', 'TERMINAL_ID', 'TX_AMOUNT',
       'TX_TIME_SECONDS', 'TX_TIME_DAYS', 'TX_FRAUD', 'day', 'month'],
      dtype='object')

In [65]:
def initiate_data_transformation():
    try:
        #logging.info('Obtaining train and test data)
        train_file_path = r'D:\iNeuron\Complete Project\Fraud Detection\Credit-Card_Fraud_Detection\artifact\data_ingestion\2023-05-02_16-13-40\train\transactions'
        test_file_path = r'D:\iNeuron\Complete Project\Fraud Detection\Credit-Card_Fraud_Detection\artifact\data_ingestion\2023-05-02_16-13-40\test\transactions'

        train_df = pd.read_csv(train_file_path)
        test_df = pd.read_csv(test_file_path)

        #logging.info('Extracting datetime and removing unnecessary columns)
        new_train_df = extract_necessary_and_drop_unncessary_columns(data=train_df)
        new_test_df = extract_necessary_and_drop_unncessary_columns(data=test_df)

        #logging.info('Obtaining preprocessing objects)
        scaler = StandardScaler()

        target = 'TX_FRAUD'

        input_train_df = new_train_df.drop(target, axis = 1)
        target_train = new_train_df[target]

        input_test_df = new_test_df.drop(target, axis= 1)
        target_test = new_test_df[target]

        columns = input_train_df.columns

        #logging.info(f"Applying preprocessing object on training dataframe and testing dataframe")
        new_scaled_train = pd.DataFrame(scaler.fit_transform(input_train_df), columns = columns)
        new_scaled_test = pd.DataFrame(scaler.transform(input_test_df), columns = columns)

        new_scaled_train[target] = target_train
        new_scaled_test[target] = target_test

        return new_scaled_train, new_scaled_test

    except Exception as e:
        raise e

In [66]:
train, test = initiate_data_transformation()